<a href="https://colab.research.google.com/github/bborisggg/hw_econometrics/blob/main/hw_econometrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Опираемся на одну из возможных спецификаций модели:

https://doi.org/10.1108/IJOEM-10-2018-0551

Берем из статьи набор переменных

$$X_t = (\Delta oilp_t ; ygap_t ; \Delta ms_t ; \Delta ir_t ; \Delta er_t; \Delta cpi_t)'$$

Отсюда берем данные по цене на нефть марки Brent
https://fred.stlouisfed.org/series/MCOILBRENTEU